# 1 - install dependencies

In [1]:
import numpy as np
import cv2 
import os
import mediapipe as mp
import tensorflow as tf 
from tensorflow import keras
import matplotlib.pyplot as plt
import time
import pandas as pd

# 2 - keypoints extractions and drawing

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

num_hand_marks = 21
num_pose_marks = 33


# holistic model process image and return the results as keypoints
def mediapipe_detection(image,model):
    image  = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image  = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image,results
    
def draw_styled_landmarks(image,results):
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
    

# read the keypoints and extract them and process them
def extract_keypoints(results):
    # extract pose marks
    if results.pose_landmarks:
        pose = np.array([ [res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark ]).flatten()
    else:
        pose = np.zeros(num_pose_marks*4)
    
    # extract left hand
    if results.left_hand_landmarks:
        left_hand = np.array([ [res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark ]).flatten()
    else:
        left_hand = np.zeros(num_hand_marks*3)
        
        
    # extract right hand
    if results.right_hand_landmarks:
        right_hand = np.array([ [res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark ]).flatten()
    else:
        right_hand = np.zeros(num_hand_marks*3)
    
    return np.concatenate([pose,left_hand,right_hand])
    

# 3 - read and process data

In [4]:
data_path = "../../../data"
train_labels = pd.read_csv(os.path.join(data_path,"train_labels.csv"),names=['sample','id'])
class_id = pd.read_csv(os.path.join(data_path,"class_id.csv"))


    
data_train_path = os.path.join(data_path,"train")



In [5]:
def check_file(file_path):
    try:
        f = open(file_path)
        f.close()
        return True
    except IOError:
        return False
    
    

def construct_path(file):
    return os.path.join(data_train_path,file+"_color.mp4")
    

### 3.1 collect data that in classes 0,1,2 and available in the data/train directory

In [6]:
def get_data(id):
    data =  train_labels[train_labels['id']==id]
    lis =  [construct_path(i) for i in  (data['sample'])]
    data =  [i for i in lis if check_file(i)]
    return data,[id for i in data]
    
    

In [7]:
actions = ['sister','hurry','hungry','enjoy_your_meal']
n_actions = len(actions)
dic = {"sister":0,"hurry":1,"hungry":2}

In [8]:


x = [] 
y = []

for i in range(n_actions):
    data,labels = get_data(i)
    x.extend(data)
    y.extend(labels)

    
train = x 
labels = y

### 3.2 process videos into frames and label to cateogries

In [9]:


def capture_frames(video_path):
    video = cv2.VideoCapture(video_path)
    video_length = int(video.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
    # count = 0
    # frame_rate = video.get(cv2.CAP_PROP_FRAME_COUNT)

    count=0
    frames = []
    while video.isOpened():
      # frame_id = video.get(0)
        ret, frame = video.read()
        
        if not ret:
            continue
        frames.append(frame)
        count += 1
        # If there are no more frames left
        if (count > (video_length-1)):
          # Release the feed
          video.release()
        # if frame_id % math.floor(1) == 0:
        #     file_name = f"{video_path}_frame_{count}.jpg"
        #     count += 1
        #     # print(f"File name: {file_name}")
        #     frames.append(frame)
        #     # cv2.imwrite(file_name, frame)
    video.release()
    return np.array(frames)


def get_frames(video_path,num_frames):
    frames = capture_frames(video_path)
    video_length = len(frames)
    steps = video_length/num_frames
    count=0
    new_frames=[]
    while count<video_length:
        frame = frames[int(count)]
#         frame = cv2.resize(frame, (256, 256))
#         frame =  (frame/255.0).astype(np.float16)
        new_frames.append(frame)
        count+=steps

    s = np.array(new_frames[:num_frames])
    print("SHAPE",s.shape)
    return np.array(new_frames[:num_frames])


In [9]:
def extract_keypoints_video(path,i=-1):
    if(i):
        print(i,end="\t")
    frames = get_frames(path,20)
    output_key_points=[]
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        for frame in frames:
            image, results = mediapipe_detection(frame, holistic)
            output_key_points.append(extract_keypoints(results))
    return np.array(output_key_points)
    

### 3.3 - extract the keypoints and save them in train

In [11]:
if "extracted_features.npy" in os.listdir():
    train = np.load("extracted_features.npy")
else:
    train = np.array([extract_keypoints_video(path,f'iteration : {i+1}/{len(x)}') for i,path in enumerate(x) ])
    np.save('extracted_features.npy', train) 

In [12]:
train.shape

(84, 20, 258)

### 3.4 split the data (training,testing)

In [13]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [14]:
labels = to_categorical(y)

In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(train, labels, test_size=0.05)

In [16]:
print(
X_train.shape,
X_test.shape,
Y_train.shape,
Y_test.shape,
)

(79, 20, 258) (5, 20, 258) (79, 3) (5, 3)


# 4 - build the model

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)



In [20]:
input_layer = Input(shape=(20,258))
layer = LSTM(64,return_sequences=True,activation="relu")(input_layer)
layer = LSTM(128,return_sequences=True,activation="relu")(layer)
layer = LSTM(64,return_sequences=False,activation="relu")(layer)
layer = Dense(64,activation="relu")(layer)
layer = Dense(32,activation="relu")(layer)
layer = Dense(len(actions),activation="softmax")(layer)

model = Model(inputs=input_layer,outputs=layer)

In [21]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 20, 258)]         0         
                                                                 
 lstm_2 (LSTM)               (None, 20, 64)            82688     
                                                                 
 lstm_3 (LSTM)               (None, 20, 128)           98816     
                                                                 
 lstm_4 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99    

In [22]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [39]:
model.fit(X_train, Y_train, epochs=200, callbacks=[tb_callback],validation_data=(X_test,Y_test))

Epoch 1/200
3/3 [==============================] - 3s 200ms/step - loss: 1.1082 - categorical_accuracy: 0.2785 - val_loss: 1.1056 - val_categorical_accuracy: 0.2000
Epoch 2/200
3/3 [==============================] - 0s 60ms/step - loss: 1.0900 - categorical_accuracy: 0.3924 - val_loss: 1.0770 - val_categorical_accuracy: 0.6000
Epoch 3/200
3/3 [==============================] - 0s 64ms/step - loss: 1.0609 - categorical_accuracy: 0.6456 - val_loss: 1.0272 - val_categorical_accuracy: 0.6000
Epoch 4/200
3/3 [==============================] - 0s 56ms/step - loss: 0.9712 - categorical_accuracy: 0.6329 - val_loss: 0.9216 - val_categorical_accuracy: 0.6000
Epoch 5/200
3/3 [==============================] - 0s 60ms/step - loss: 0.8425 - categorical_accuracy: 0.5949 - val_loss: 0.5556 - val_categorical_accuracy: 0.6000
Epoch 6/200
3/3 [==============================] - 0s 56ms/step - loss: 0.5769 - categorical_accuracy: 0.8228 - val_loss: 0.4223 - val_categorical_accuracy: 0.8000
Epoch 7/200
3/3

Epoch 51/200
3/3 [==============================] - 0s 60ms/step - loss: 0.4058 - categorical_accuracy: 0.8228 - val_loss: 0.4853 - val_categorical_accuracy: 0.6000
Epoch 52/200
3/3 [==============================] - 0s 62ms/step - loss: 0.3945 - categorical_accuracy: 0.7342 - val_loss: 0.3010 - val_categorical_accuracy: 0.8000
Epoch 53/200
3/3 [==============================] - 0s 59ms/step - loss: 0.2660 - categorical_accuracy: 0.9494 - val_loss: 0.2680 - val_categorical_accuracy: 1.0000
Epoch 54/200
3/3 [==============================] - 0s 60ms/step - loss: 0.3548 - categorical_accuracy: 0.8734 - val_loss: 0.2182 - val_categorical_accuracy: 1.0000
Epoch 55/200
3/3 [==============================] - 0s 61ms/step - loss: 0.2752 - categorical_accuracy: 0.9114 - val_loss: 0.1939 - val_categorical_accuracy: 1.0000
Epoch 56/200
3/3 [==============================] - 0s 61ms/step - loss: 0.1864 - categorical_accuracy: 0.9367 - val_loss: 0.0998 - val_categorical_accuracy: 1.0000
Epoch 57/2

Epoch 101/200
3/3 [==============================] - 0s 62ms/step - loss: 0.3722 - categorical_accuracy: 0.7342 - val_loss: 0.4879 - val_categorical_accuracy: 0.6000
Epoch 102/200
3/3 [==============================] - 0s 60ms/step - loss: 0.3577 - categorical_accuracy: 0.7342 - val_loss: 0.3857 - val_categorical_accuracy: 0.8000
Epoch 103/200
3/3 [==============================] - 0s 63ms/step - loss: 0.3350 - categorical_accuracy: 0.9114 - val_loss: 0.3632 - val_categorical_accuracy: 0.8000
Epoch 104/200
3/3 [==============================] - 0s 62ms/step - loss: 0.2874 - categorical_accuracy: 0.8608 - val_loss: 0.4058 - val_categorical_accuracy: 0.8000
Epoch 105/200
3/3 [==============================] - 0s 61ms/step - loss: 0.4858 - categorical_accuracy: 0.8734 - val_loss: 0.3559 - val_categorical_accuracy: 1.0000
Epoch 106/200
3/3 [==============================] - 0s 64ms/step - loss: 0.9580 - categorical_accuracy: 0.8987 - val_loss: 1.8097 - val_categorical_accuracy: 0.4000
Epoc

3/3 [==============================] - 0s 56ms/step - loss: 0.4657 - categorical_accuracy: 0.6835 - val_loss: 0.4060 - val_categorical_accuracy: 0.6000
Epoch 200/200
3/3 [==============================] - 0s 58ms/step - loss: 0.4642 - categorical_accuracy: 0.6582 - val_loss: 0.3963 - val_categorical_accuracy: 0.8000


In [40]:
model.fit(X_train, Y_train, epochs=400, callbacks=[tb_callback],validation_data=(X_test,Y_test))

Epoch 1/400
3/3 [==============================] - 0s 67ms/step - loss: 0.4638 - categorical_accuracy: 0.6582 - val_loss: 0.3963 - val_categorical_accuracy: 0.8000
Epoch 2/400
3/3 [==============================] - 0s 56ms/step - loss: 0.4644 - categorical_accuracy: 0.6582 - val_loss: 0.3974 - val_categorical_accuracy: 0.8000
Epoch 3/400
3/3 [==============================] - 0s 55ms/step - loss: 0.4642 - categorical_accuracy: 0.6835 - val_loss: 0.4005 - val_categorical_accuracy: 0.6000
Epoch 4/400
3/3 [==============================] - 0s 57ms/step - loss: 0.4610 - categorical_accuracy: 0.6962 - val_loss: 0.3902 - val_categorical_accuracy: 0.8000
Epoch 5/400
3/3 [==============================] - 0s 55ms/step - loss: 0.4596 - categorical_accuracy: 0.6582 - val_loss: 0.3816 - val_categorical_accuracy: 0.8000
Epoch 6/400
3/3 [==============================] - 0s 58ms/step - loss: 0.4591 - categorical_accuracy: 0.6582 - val_loss: 0.3896 - val_categorical_accuracy: 0.8000
Epoch 7/400
3/3 

Epoch 51/400
3/3 [==============================] - 0s 63ms/step - loss: 0.3363 - categorical_accuracy: 0.6709 - val_loss: 0.3692 - val_categorical_accuracy: 0.6000
Epoch 52/400
3/3 [==============================] - 0s 61ms/step - loss: 0.3600 - categorical_accuracy: 0.6709 - val_loss: 0.3375 - val_categorical_accuracy: 0.6000
Epoch 53/400
3/3 [==============================] - 0s 58ms/step - loss: 0.3126 - categorical_accuracy: 0.7468 - val_loss: 0.3920 - val_categorical_accuracy: 1.0000
Epoch 54/400
3/3 [==============================] - 0s 64ms/step - loss: 0.4313 - categorical_accuracy: 0.7975 - val_loss: 0.2898 - val_categorical_accuracy: 1.0000
Epoch 55/400
3/3 [==============================] - 0s 62ms/step - loss: 0.2792 - categorical_accuracy: 0.9367 - val_loss: 0.3658 - val_categorical_accuracy: 0.6000
Epoch 56/400
3/3 [==============================] - 0s 59ms/step - loss: 0.2423 - categorical_accuracy: 0.9241 - val_loss: 0.2792 - val_categorical_accuracy: 1.0000
Epoch 57/4

Epoch 101/400
3/3 [==============================] - 0s 63ms/step - loss: 0.5976 - categorical_accuracy: 0.6582 - val_loss: 0.5289 - val_categorical_accuracy: 0.8000
Epoch 102/400
3/3 [==============================] - 0s 53ms/step - loss: 0.5833 - categorical_accuracy: 0.6582 - val_loss: 0.5300 - val_categorical_accuracy: 0.6000
Epoch 103/400
3/3 [==============================] - 0s 60ms/step - loss: 0.5684 - categorical_accuracy: 0.6582 - val_loss: 0.5275 - val_categorical_accuracy: 0.6000
Epoch 104/400
3/3 [==============================] - 0s 58ms/step - loss: 0.5616 - categorical_accuracy: 0.6456 - val_loss: 0.5181 - val_categorical_accuracy: 0.6000
Epoch 105/400
3/3 [==============================] - 0s 63ms/step - loss: 0.5540 - categorical_accuracy: 0.6582 - val_loss: 0.5035 - val_categorical_accuracy: 0.8000
Epoch 106/400
3/3 [==============================] - 0s 60ms/step - loss: 0.5466 - categorical_accuracy: 0.6582 - val_loss: 0.4885 - val_categorical_accuracy: 0.8000
Epoc

3/3 [==============================] - 0s 58ms/step - loss: 0.6604 - categorical_accuracy: 0.5823 - val_loss: 0.4097 - val_categorical_accuracy: 0.8000
Epoch 200/400
3/3 [==============================] - 0s 55ms/step - loss: 0.6429 - categorical_accuracy: 0.5823 - val_loss: 0.4013 - val_categorical_accuracy: 0.8000
Epoch 201/400
3/3 [==============================] - 0s 67ms/step - loss: 0.6252 - categorical_accuracy: 0.5823 - val_loss: 0.3727 - val_categorical_accuracy: 0.8000
Epoch 202/400
3/3 [==============================] - 0s 57ms/step - loss: 0.5976 - categorical_accuracy: 0.5823 - val_loss: 0.3461 - val_categorical_accuracy: 0.8000
Epoch 203/400
3/3 [==============================] - 0s 66ms/step - loss: 0.5567 - categorical_accuracy: 0.6203 - val_loss: 0.2904 - val_categorical_accuracy: 0.8000
Epoch 204/400
3/3 [==============================] - 0s 56ms/step - loss: 0.5192 - categorical_accuracy: 0.6962 - val_loss: 0.2416 - val_categorical_accuracy: 0.8000
Epoch 205/400
3/3 

3/3 [==============================] - 0s 63ms/step - loss: 0.0135 - categorical_accuracy: 1.0000 - val_loss: 0.0103 - val_categorical_accuracy: 1.0000
Epoch 298/400
3/3 [==============================] - 0s 55ms/step - loss: 0.0111 - categorical_accuracy: 1.0000 - val_loss: 0.0081 - val_categorical_accuracy: 1.0000
Epoch 299/400
3/3 [==============================] - 0s 56ms/step - loss: 0.0086 - categorical_accuracy: 1.0000 - val_loss: 0.0066 - val_categorical_accuracy: 1.0000
Epoch 300/400
3/3 [==============================] - 0s 64ms/step - loss: 0.0071 - categorical_accuracy: 1.0000 - val_loss: 0.0054 - val_categorical_accuracy: 1.0000
Epoch 301/400
3/3 [==============================] - 0s 59ms/step - loss: 0.0060 - categorical_accuracy: 1.0000 - val_loss: 0.0046 - val_categorical_accuracy: 1.0000
Epoch 302/400
3/3 [==============================] - 0s 67ms/step - loss: 0.0051 - categorical_accuracy: 1.0000 - val_loss: 0.0039 - val_categorical_accuracy: 1.0000
Epoch 303/400
3/3 

3/3 [==============================] - 0s 69ms/step - loss: 2.1951e-04 - categorical_accuracy: 1.0000 - val_loss: 1.8019e-04 - val_categorical_accuracy: 1.0000
Epoch 393/400
3/3 [==============================] - 0s 54ms/step - loss: 2.1646e-04 - categorical_accuracy: 1.0000 - val_loss: 1.7649e-04 - val_categorical_accuracy: 1.0000
Epoch 394/400
3/3 [==============================] - 0s 65ms/step - loss: 2.1187e-04 - categorical_accuracy: 1.0000 - val_loss: 1.7306e-04 - val_categorical_accuracy: 1.0000
Epoch 395/400
3/3 [==============================] - 0s 57ms/step - loss: 2.0806e-04 - categorical_accuracy: 1.0000 - val_loss: 1.6970e-04 - val_categorical_accuracy: 1.0000
Epoch 396/400
3/3 [==============================] - 0s 55ms/step - loss: 2.0479e-04 - categorical_accuracy: 1.0000 - val_loss: 1.6632e-04 - val_categorical_accuracy: 1.0000
Epoch 397/400
3/3 [==============================] - 0s 64ms/step - loss: 2.0182e-04 - categorical_accuracy: 1.0000 - val_loss: 1.6310e-04 - val

In [41]:
model.save_weights("model.h5")

In [23]:
model.load_weights('model.h5')

# 5 - Make predictions

### this doesn't make any sence

In [21]:
model.evaluate(X_test,Y_test)

1/1 [==============================] - 1s 1s/step - loss: 5.1577e-04 - categorical_accuracy: 1.0000


[0.0005157673731446266, 1.0]

In [23]:
model.evaluate(X_train,Y_train)

3/3 [==============================] - 0s 8ms/step - loss: 1.9180e-04 - categorical_accuracy: 1.0000


[0.0001918047055369243, 1.0]

In [48]:
x[0]

'../../data/train/signer0_sample431_color.mp4'

# 6 - test in real time

In [24]:
from scipy import stats

colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame


In [25]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()
        if(not ret):
            break
#         frame = cv2.resize(frame, (512, 512))

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-20:]
        
        if len(sequence) == 20:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-2:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [60]:
cap.release()

In [61]:
cv2.destroyAllWindows()